In [58]:
#BRADLEY SCOTT
# this cell is just auto formatting so I can be lazy and still have pretty code
#!pip install black[jupyter]

#from google.colab import drive
#drive.mount("/content/drive")

#!black /content/drive/MyDrive/'Colab Notebooks'/'DATA602_HW2.ipynb'

All done! ✨ 🍰 ✨
1 file left unchanged.


In [ ]:
#NOTE:
# I used chatgpt quite a bit. This is maybe the second ML model I've ever trained.
# I put a lot of effort into making sure I understood the process and changes being
# suggested.

In [59]:
import pandas as pd

# I saved the data in my google drive so I could acccess it here
file_path = "/content/drive/My Drive/finefoods_training.txt"

# create an empty list to store the parsed data
data = []

# use 'ISO-8859-1' encoding instead of the default 'utf-8'
# the utf-8 default encoding ran into issues
with open(file_path, "r", encoding="ISO-8859-1") as file:
    review = {}
    for line in file:
        line = line.strip()
        if line:
            key, value = line.split(": ", 1)
            review[key] = value
        else:
            data.append(review)
            review = {}

df = pd.DataFrame(data)
# print(df.head())

# wanted to see how many rows there is
# num_rows = df.shape[0]
# print(num_rows)

# what's the average size of review/text - 401.366
# lengths = df['review/text'].apply(len)
# average_length = lengths.mean()
# print(average_length)

# what's the average size of review/summary - 22.8375
# lengths = df['review/summary'].apply(len)
# average_length = lengths.mean()
# print(average_length)

# NOTE: THIS WAS ME JUST INITIALLY LOOKING AT THE DATA AND WHAT NOT.
# I IMPORT THE DATA AGAIN WHEN I CONSTRUCT THE ML MODELS
# BECAUSE I WAS MAKING CHANGES TO THE WAY THAT I PROCESS THEM

In [60]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

# make vectors from review/summary and review/text
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 3))
summary_features = tfidf.fit_transform(df["review/summary"])
text_features = tfidf.fit_transform(df["review/text"])

encoder = LabelEncoder()
userId_encoded = encoder.fit_transform(df["review/userId"])
profileName_encoded = encoder.fit_transform(df["review/profileName"])

# combine the features and define the target variable
from scipy.sparse import hstack

X = hstack(
    [
        summary_features,
        text_features,
        userId_encoded[:, None],
        profileName_encoded[:, None],
    ]
)
y = df["review/score"]

# train the model
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train, y_train)

from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, zero_division=0))

# NOTE: This initial prediction is really only ok for category 5
# which indicates an imbalance in the data. The next cell is the culimination of
# me trying to find ways to make a better model that works well for more than just
# category 5, but it required using other methods than Naive Bayes

Accuracy: 0.5
              precision    recall  f1-score   support

         1.0       0.07      0.12      0.09        40
         2.0       0.00      0.00      0.00        22
         3.0       0.00      0.00      0.00        27
         4.0       0.20      0.16      0.18        49
         5.0       0.65      0.71      0.68       262

    accuracy                           0.50       400
   macro avg       0.18      0.20      0.19       400
weighted avg       0.46      0.50      0.48       400



In [61]:
# loading google pre-trained data to use in our ML model
import gensim.downloader as api

# Load Google's pre-trained Word2Vec model.
w2v_model = api.load("word2vec-google-news-300")

In [62]:
#!pip install spacy
#!pip install imbalanced-learn
#!pip install gensim

# import needed modules
import pandas as pd
import spacy
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from scipy.sparse import hstack

# load spacy model
nlp = spacy.load("en_core_web_sm")


# spacy preprocessing
def spacy_preprocess(text):
    doc = nlp(text)
    lemmatized = [
        token.lemma_ for token in doc if not token.is_stop and not token.is_punct
    ]
    return " ".join(lemmatized)


# document vector creation
def document_vector(word_list):
    # remove out-of-vocabulary words
    word_list = [word for word in word_list if word in w2v_model.key_to_index]
    if len(word_list) == 0:
        return np.zeros(300)  # Word2Vec uses 300
    return np.mean(w2v_model[word_list], axis=0)


# Read the data
file_path = "/content/drive/My Drive/finefoods_training.txt"
data = []

with open(file_path, "r", encoding="ISO-8859-1") as file:
    review = {}
    for line in file:
        line = line.strip()
        if line:
            key, value = line.split(": ", 1)
            review[key] = value
        else:
            data.append(review)
            review = {}
df = pd.DataFrame(data)

df["processed_summary"] = df["review/summary"].apply(spacy_preprocess)
df["processed_text"] = df["review/text"].apply(spacy_preprocess)

# create word2vec features
df["w2v_summary"] = df["processed_summary"].apply(lambda x: document_vector(x.split()))
df["w2v_text"] = df["processed_text"].apply(lambda x: document_vector(x.split()))

# encode other categorical encoders
encoder = LabelEncoder()
userId_encoded = encoder.fit_transform(df["review/userId"])
profileName_encoded = encoder.fit_transform(df["review/profileName"])

# Combine all features
X = np.hstack(
    [
        np.array(df["w2v_summary"].tolist()),
        np.array(df["w2v_text"].tolist()),
        userId_encoded[:, None],
        profileName_encoded[:, None],
    ]
)

# Target variable
y = df["review/score"].astype("category")

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Apply SMOTE to the training data
# SMOTE is used to try and correct for imbalances in the data
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Train the model with RandomForest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_smote, y_train_smote)

# Make predictions and evaluate the model
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


# NOTE: Notice how the f1-score is better for the other classifications now.
# overall, it's still not great with an accuracy of 0.645. It would likely require
# a lot more tinkering for me to get the accuracy greatly improved.

Accuracy: 0.645
              precision    recall  f1-score   support

         1.0       0.48      0.38      0.42        40
         2.0       0.57      0.18      0.28        22
         3.0       0.17      0.11      0.13        27
         4.0       0.08      0.04      0.05        49
         5.0       0.74      0.89      0.81       262

    accuracy                           0.65       400
   macro avg       0.41      0.32      0.34       400
weighted avg       0.58      0.65      0.60       400



In [63]:
# Optional Task
# predict the review/helpfulness form the rest of the information

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import spacy
from scipy.sparse import hstack
import numpy as np
from datetime import datetime


# function to preprocess text with spacy
def spacy_preprocess(text, nlp_model):
    doc = nlp_model(text)
    return " ".join(
        [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    )


# function to convert helpfulness to ratio
# I don't think predicting the helpfulness numbers
# themselves (so like 10/10 or 7/8) would be possible to get a good prediction of
# and I don't think it's as useful as the actual helpfulness ratio
def helpfulness_ratio(helpfulness):
    num, denom = map(int, helpfulness.split("/"))
    if denom == 0:
        return np.nan  # Return NaN for 0 denominators
    return num / denom


# load the spacy model
nlp = spacy.load("en_core_web_sm")

# load the dataset
file_path = "/content/drive/My Drive/finefoods_training.txt"
data = []

with open(file_path, "r", encoding="ISO-8859-1") as file:
    review = {}
    for line in file:
        line = line.strip()
        if line:
            key, value = line.split(": ", 1)
            review[key] = value
        else:
            data.append(review)
            review = {}

# create DataFrame
df = pd.DataFrame(data)

# convert time to a datetime object and then to a numerical feature
df["review/time"] = pd.to_datetime(df["review/time"].astype(int), unit="s")
latest_review_time = df["review/time"].max()
df["review_time_diff"] = (latest_review_time - df["review/time"]).dt.days

# preprocess text data
df["processed_summary"] = df["review/summary"].apply(lambda x: spacy_preprocess(x, nlp))
df["processed_text"] = df["review/text"].apply(lambda x: spacy_preprocess(x, nlp))

# convert helpfulness to ratio.
df["helpfulness_ratio"] = df["review/helpfulness"].apply(helpfulness_ratio)

# drop rows with NaN helpfulness_ratio
df.dropna(subset=["helpfulness_ratio"], inplace=True)

# vectorize text data
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
summary_features = tfidf.fit_transform(df["processed_summary"])
text_features = tfidf.fit_transform(df["processed_text"])

# encode categorical features and scale numerical features
encoder = LabelEncoder()
userId_encoded = encoder.fit_transform(df["review/userId"])
profileName_encoded = encoder.fit_transform(df["review/profileName"])
productId_encoded = encoder.fit_transform(df["product/productId"])
score_encoded = encoder.fit_transform(df["review/score"])
scaler = StandardScaler()
# adjust for the time scale (mentioned in project description)
review_time_diff_scaled = scaler.fit_transform(df[["review_time_diff"]])

# combine the features
X = hstack(
    [
        summary_features,
        text_features,
        userId_encoded[:, None],
        profileName_encoded[:, None],
        productId_encoded[:, None],
        score_encoded[:, None],
        review_time_diff_scaled,
    ]
)
y = df["helpfulness_ratio"]

# split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# train the model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# make predictions and check MSE and R squared
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

mse, r2

# NOTE: THe MSE and R squared values are not great. We changed the review/helpfulness
# to a ratio so it's between 0 and 1. An MSE of 0.1 is pretty high for this range.
# The R squared value means that we can only explain about 20% of the variance
# from the model. I didn't think this would be an easy thing to model since
# a lot of the reviews are going to have 0/0 for review/helpfulness without
# any obvious reason why

(0.10315733619839082, 0.2012137219861615)